# From Millions of Seconds to Milliseconds: a premier on Python high-performance programming

Zach Lowry

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### AHE: Adaptive histogram equalization

From [Wikipedia](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization):

 * AHE is a computer [image processing](https://en.wikipedia.org/wiki/Image_processing) technique used to improve [contrast](https://en.wikipedia.org/wiki/Contrast_(vision)) in images. 
 * It differs from ordinary [histogram equalization](https://en.wikipedia.org/wiki/Histogram_equalization) in the respect that the adaptive method computes several [histograms](https://en.wikipedia.org/wiki/Histogram), each corresponding to a distinct section of the image, and uses them to redistribute the lightness values of the image. 
 * It is therefore suitable for improving the local contract and enhancing the definitions of edges in each region of an image. 
    


### CLAHE: Contrast-limited adaptive histogram equalization

Continued from [Wikipedia](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization):

 * AHE has a tendency to overamplify noise in relatively homogeneous regions of an image. 
 * A variant of adaptive histogram equalization called contrast limited adaptive histogram equalization (CLAHE) prevents this by limiting the amplification.



### FLCLAHE: Fully-localized contrast-limited adaptive histogram equalization

 * CLAHE can product uneven effects in images with very low contrast due to the histograms being re-used for each pixel in a window of pixels.
 * A more accurately enhanced image can be produced by producing a unique histogram for each pixel in the source image and calculating the cumulative distribution function on a per-pixel basis.
 * Such an algorithm would require considerable optimaization and tunint to execute in an efficient manner.

### Why bother with a Python Implementation?

 * Implementing this algorithm in Python could provide a reasonable pseudo-code-ish implementation for other implementations to be developed.
 * From this initial pure-Python implementation, subsequent implementatiosn could be refined to optimize performance or port to other frameworks and languages. 
 * The purely-parallel, shared-nothing architecture of the problem provides an ideal use case for demonstrating parallel processing performance optimization techniques.

### FLCLAHE reference implementation signature
```python
def flclahe(
    img: np.ndarray,     # a 2D NumPy array containing the image data
    clip_limit: int = 8, # the maximum # of pixel counts to use for each pixel histogram bin
    window: int = 64,    # the size in pixels of the square window around the target pixel to use
                         # when calculating the pixel histogram
    nbins: int = 256     # the # of bins to use when calculating the pixel histogram
) -> np.ndarray:
```

We will provide a reference implementation signature for our prototypical FLCLAHE implementation. Each implementation will accept as its arguments:

 * img (np.ndarray): a 2D 8-bit or 16-bit (np.uint8 or np.uint16) NumPy array containing the image data
 * clip_limit (int): an optional integer value for the maximum # of pixel counts to use for each pixel histogram bin
 * window (int): an optional integer value for the size in pixels of the square window around the target pixel to use when calculating the pixel histogram
 * nbins (int): an optional integer value for the # of bins to use when calculating the pixel histogram
 
The reference implementation should be expected to return a Numpy array of the same dtype and size as the source img.

```python
# create the output array
img_out = np.empty_like(img)

# scale the provided clip limit by the window size
clip_limit = int(ceil((clip_limit * (window ** 2)) / nbins))
```

First, we sanitize our inputs. We create an empty output array (using empty_like to skip initialization) and scale our provided clip_limit value by the window size. 

```python
# loop through each row in the image
for i in range(img.shape[0]):
    
    # calculate left index of the window, ensuring that it stays within the image boundaries
    il = i - (window // 2)

    # ensure that the left is never less than 0
    if il < 0:
        il = 0

    # also ensure that the left is not greater than one window size from the edge
    elif il >= img.shape[0] - window:
        il = img.shape[0] - window

    # add the window size to the left index to get the right index
    ir = il + window
```

Then we begin looping through each row in the image. 

We start by calculating the 'left' and 'right' boundaries of our histogram window for this row, ensuring that the 'left' side is greater than 0 and at least one window from the outer edge. 

```python
# loop through each column in the image
for j in range(img.shape[1]):

    # calculate left index of the window, ensuring that it stays within the image boundaries
    jl = j - (window // 2)

    # ensure that the left is never less than 0
    if jl < 0:
        jl = 0

    # also ensure that the left is not greater than one window size from the edge
    elif jl >= img.shape[1] - window:
        jl = img.shape[1] - window

    # add the window size to the left index to get the right index
    jr = jl + window
```

Then we loop over each column of the image and calculate the 'left' and 'right' window indexes in the same way. 

```python
# get the window region from the image
region = img[il:ir, jl:jr]

# initialize an empty array for the histogram
hist = np.zeros(shape=(nbins, ), dtype=np.uint16)
```

Next, we get the 'region' of our histogram by applying our calculated left and right index values for i and j. 

We then initialize an NumPy array to hold our histogram with zeros. 

```python
# calculate which bin this pixel should be placed in
binned_value = ((img[i, j] - img.min()) * nbins) // (img.max() - img.min())

# ensure that the binned_value is greater than 0
if binned_value < 0:
    binned_value = 0

# also ensure that the binned_value is less than nbins
elif binned_value >= nbins:
    binned_value = nbins - 1
```

We next calculate which histogram 'bin' our current pixel would belong in. 

We also perform a sanity check to ensure that the calculated 'bin' is within the limits of the histogram array. 

```python
# loop through each pixel in the region
for ri in range(region.shape[0]):
    for rj in range(region.shape[1]):

        # calculate which bin this pixel should be placed in
        b = ((region[ri, rj] - img.min()) * nbins) // (img.max() - img.min())

        # ensure that the value of b is greater than 0
        if b < 0:
            b = 0

        # also ensure that the value of b is less than nbins
        elif b >= nbins:
            b = nbins - 1

        # increment the value for the histogram for this bin
        hist[b] += 1
```

We loop through each pixel within the histogram region, performing the same 'bin' value calculation as we did before. 

We increment the count inside the histogram array for each calculated 'bin' value. 

```python
# the pixels clipped to be redistributed across all bins
excess = 0

# the calculated look up table value
lut = 0
```

Now that the histogram is calculated, we initialize some temporary values for the calculate LUT value for this pixel, and the values in excess of the clip limit that were discarded. 

```python
# loop through each histogram bin
for ii in range(nbins):

    # if our bin is less than the image pixel intensity, we will increment the "lut"
    if ii <= binned_value:

        # if the histogram bin count is greater than clip_limit, add clip limit
        if hist[ii] > clip_limit:
            lut += clip_limit

        # otherwise add the histogram value to the "lut"
        else:
            lut += hist[ii]

    # if the histogram bin count is greater than the clip limit, add save the excess
    if hist[ii] > clip_limit:
        excess += (hist[ii] - clip_limit)
```

Now, we loop through each histogram 'bin' index. If the index is less than our pixel's bin index, then we will add the histogram count to the LUT variable. 

If the histogram count is greater than the clip limit, then we add the clip limit instead of the histogram count. 

Finally, if the histogram count was greater than the clip limit, we add the remainder of the histogram count and the clip limit to the excess variable. 

```python
# start the new value with the value of the "lut"
new_value = lut

# multiply the new value by the number of histogram bins minus one
new_value *= (nbins - 1)

# add the excess "counts" times the image pixel bin value
new_value += (excess * binned_value)

# divide the new value by the number of pixels in the window
new_value //= (window ** 2)

# write the new value ot the image
img_out[i, j] = new_value
```

Lastly, we calculate a new value for our pixel. 

We initialize the new value to the LUT value, then we multiply that value by the maximum bin index. 

We add the excess counts times the pixel's binned value. 

We then divide the new pixel value by the window size squared. 

Finally, we write the new pixel value out to the output image. 

```python
# return the processed image
return img_out
```

Lastly, we return the processed image after looping through every pixel. 